# Network Deliberation ABM
## Discrete state, generated signal task

In [ ]:
# Configuration

N = 100
M = 5
runs = 100
stages = 25
steps = 1

bit_count = 7
p_error = 0.4

small_world_k = 4
small_world_a = 0.1
erdos_renyi_p = 0.01
barabasi_albert_m = 2

In [ ]:
# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})

# Imports
import math
import random

import networkx as nx
import numpy as np
from numpy import random as nprand
from tqdm.notebook import tqdm

import netdelib.soclearn as slearn
import netdelib.soclearn.evaluate as sleval
import netdelib.soclearn.models.generated as slgen
import netdelib.soclearn.strategy as slstrat
import netdelib.topologies.topologies as topo

## Network Topologies

In [ ]:
class NetworkFactory(object):
    """Base class for network factories.
    
    Constructor paramteters
    N: Number of participants
    M: Number of participants per group
    stage_graphs: Whether to regenerate network at each stage
    """

    def __init__(self, N, M, stage_graphs=True):
        self.N = N
        self.M = M
        self.stage_graphs = stage_graphs
    
    def create(self, stage):
        """Create a networkx Graph with a particular topology.
        
        Parameters
        stage: The deliberation stage
        
        Returns
        A networkx Graph.
        """
        return nx.Graph()

class LongPathFactory(NetworkFactory):
    '''Factory class for long-path networks.'''
    def create(self, stage):
        groups = topo.get_long_path_stage_groups(self.N, self.M, stage)
        G = nx.Graph()
        for group in groups:
            g = nx.complete_graph(group) #clique network
            G = nx.union(G, g)
        return G
    
class RandomGroupFactory(NetworkFactory):
    """Factory class for random group networks."""
    def create(self, stage):
        groups = topo.get_random_stage_groups(self.N, self.M, stage)
        G = nx.Graph()
        for group in groups:
            g = nx.complete_graph(group) #clique network
            G = nx.union(G, g)
        return G

class CompleteFactory(NetworkFactory):
    """Factory class for deliberation network with random group assignments."""

    def __init__(self, N, M):
        super(CompleteFactory, self).__init__(N, M, False)
        
    def create(self, stage):
        return nx.complete_graph(self.N)

class PreferentialFactory(NetworkFactory):
    """Factory class for preferential attachment networks.
    
    Constructor Params
    N: Number of participants
    M: Number of participants per group (unused)
    m: Number of edges per node"""
    
    def __init__(self, N, M, m):
        super(PreferentialFactory, self).__init__(N, M, False)
        self.m = m
        
    def create(self, stage):
        return nx.barabasi_albert_graph(self.N, self.m)
    
class SmallWorldFactory(NetworkFactory):
    """Factory class for small-world networks.
    
    Constructor Params
    N: Number of participants
    M: Number of participants per group (unused)
    k: Number of edges per node
    a: probability of rewiring
    """
    
    def __init__(self, N, M, k, a):
        super(SmallWorldFactory, self).__init__(N, M, False)
        self.k = k
        self.a = a
        
    def create(self, stage):
        return nx.watts_strogatz_graph(self.N, self.k, self.a)
    
class RandomFactory(NetworkFactory):
    """Factory class for Erdos-Renyi random networks.
    
    Constructor Params
    N: Number of participants
    M: Number of participants per group (unused)
    p: Probability of edge existing between two nodes
    """
    
    def __init__(self, N, M, p):
        super(RandomFactory, self).__init__(N, M, False)
        self.p
        
    def create(self, stage):
        return nx.erdos_renyi_graph(N, self.p)


## ABM Simulation

In [ ]:
def run_trial(
    factory,
    p_error,
    learning_strategy,
    true_value,
    N,
    M,
    stages,
    steps
):
    """Run a single trial
    
    Parameters:
    factory: network factory
    learning strategy: learning strategy to simulate in topology
    intial beliefs are generated inside the topology: prarms (G, true_value, p_error)
    true_value: the ground truth 
    stages: the number of stages in each trial
    steps: the number of learning steps per stage
    
    Returns
    A list of dictionaries, one for each time step.
    Each dictionary maps collaborator ids to belief states.
    """
    beliefs_stages = []
    for stage in range(stages):
        if stage == 0:
            # Create new network and initial beliefs at stage 0
            G = factory.create(stage)
            beliefs = slgen.initial_beliefs_noisy(G, true_value, p_error=p_error)
            beliefs_stages.append(beliefs)
        else:
            # At later stages, only create new network if factory.stage_graphs is True
            if factory.stage_graphs:
                G = factory.create(stage)

        # Run several learning steps and add beliefs at each step to beliefs_stages
        # The first element of the result is just the initial belief, which is already in beliefs_stages
        beliefs_list = slearn.learn(G, beliefs_stages[-1], learning_strategy, true_value, steps)
        beliefs_stages += beliefs_list[1:]
    return beliefs_stages

# Run Simulation 

In [ ]:
networks = {
    'Complete': CompleteFactory(N, M),
    'Pref. Attach.': PreferentialFactory(N, M, barabasi_albert_m),
    'Small World': SmallWorldFactory(N, M, small_world_k, small_world_a),
    'Long Path': LongPathFactory(N, M),
    'Random Group': RandomGroupFactory(N, M)   
}

In [ ]:
learning_strategies = {
    'Best Neighbor': slstrat.best_neighbor,
#    'Random bit': random_neighbor_bit,
#    'Random list': random_neighbor_list,
    'Conform': slstrat.conform,
    'Local Majority': slstrat.local_majority,
}

In [ ]:
num_net = len(networks)
num_strat = len(learning_strategies)
true_value = tuple([1 for bit in range(bit_count)])

In [ ]:
score_fraction = [[0 for strat in range(num_net)] for net in range(num_strat)]
score_distance = [[0 for strat in range(num_net)] for net in range(num_strat)]

for i, (title, learning_strategy) in enumerate(tqdm(learning_strategies.items())):
    for j, (title2, factory) in enumerate(networks.items()):
        
        # Record time series for fraction correct
        run_fraction = []
        for run in range(runs):
            beliefs = run_trial(
                factory,
                p_error,
                learning_strategy,
                true_value,
                N, M, stages, steps)
            run_fraction.append(sleval.beliefs_correct(beliefs, true_value))
        mean = np.mean(run_fraction, axis=0)
        std = np.std(run_fraction, axis=0)
        score_fraction[i][j] = (mean, std)



In [ ]:
plt.figure(figsize=(15, 9))
for i, (title, learning_strategy) in enumerate(learning_strategies.items()):
    for j, (title2, factory) in enumerate(networks.items()):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)
        y = score_fraction[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score_fraction[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaaa")
        plt.plot(range(stages * steps + 1), y)        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Correct')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.tight_layout()
        plt.ylim([0, 1])
